In [56]:
import sys
import os
# Navigate up one level to the parent directory and append it to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
import nfl_data_py as nfl
import pandas as pd
from src import utils
from src.config import DATA_PATH, PFF_PROP_PATH


pd.set_option('display.max_columns', None)  # Display all columns

In [57]:
import importlib

importlib.reload(utils)

<module 'src.utils' from '/Users/fgreichert/Documents/code/nfl-betting/src/utils.py'>

In [3]:
year = 2023
week = 3

In [4]:
# Read in PFF Prop Data
pff_df = pd.read_csv(PFF_PROP_PATH /f'nfl-best-bets-2023-week-{week}.csv')
pff_df.shape

(728, 17)

In [5]:
pff_df.head()

,propType,player,position,team,opponent,start,line,sideOneType,sideOneOdds,sideOneValue,sideOneCash,sideOneTickets,sideTwoType,sideTwoOdds,sideTwoValue,sideTwoCash,sideTwoTickets
0,pass_att,G. Minshew,QB,IND,BLT,2023-09-24T17:00:00Z,32.5,over,-132,-0.511813,NaN,NaN,under,-102,0.437888,NaN,NaN
1,pass_att,A. Dalton,QB,CAR,SEA,2023-09-24T20:05:00Z,31.5,over,-124,-0.500065,NaN,NaN,under,-108,0.427262,NaN,NaN
2,pass_comp,A. Dalton,QB,CAR,SEA,2023-09-24T20:05:00Z,19.5,over,-117,-0.493363,NaN,NaN,under,-115,0.419308,NaN,NaN
3,pass_comp,G. Minshew,QB,IND,BLT,2023-09-24T17:00:00Z,20.5,over,-114,-0.480002,NaN,NaN,under,-118,0.405990,NaN,NaN
4,recv_rec,C. Hubbard,RB,CAR,SEA,2023-09-24T20:05:00Z,1.5,over,-195,-0.355043,NaN,NaN,under,143,0.282509,NaN,NaN


In [10]:
edges = []
overs = []
implied = []

for row in pff_df.itertuples():
    side = 'One' if row.sideOneValue > row.sideTwoValue else 'Two'

    # Use getattr to access column values dynamically
    side_value = getattr(row, f'side{side}Value')
    side_type = getattr(row, f'side{side}Type')
    side_odds = getattr(row, f'side{side}Odds')

    edges.append(side_value)
    overs.append(True if side_type == 'over' else False)
    implied.append(utils.implied_probability(side_odds))

pff_df['pff_edge'] = edges
pff_df['over_bet'] = overs
pff_df['implied_prob'] = implied

In [11]:
pff_df.head()

,propType,player,position,team,opponent,start,line,sideOneType,sideOneOdds,sideOneValue,sideOneCash,sideOneTickets,sideTwoType,sideTwoOdds,sideTwoValue,sideTwoCash,sideTwoTickets,pff_edge,over_bet,implied_prob
0,pass_att,G. Minshew,QB,IND,BLT,2023-09-24T17:00:00Z,32.5,over,-132,-0.511813,NaN,NaN,under,-102,0.437888,NaN,NaN,0.437888,0,0.5050
1,pass_att,A. Dalton,QB,CAR,SEA,2023-09-24T20:05:00Z,31.5,over,-124,-0.500065,NaN,NaN,under,-108,0.427262,NaN,NaN,0.427262,0,0.5192
2,pass_comp,A. Dalton,QB,CAR,SEA,2023-09-24T20:05:00Z,19.5,over,-117,-0.493363,NaN,NaN,under,-115,0.419308,NaN,NaN,0.419308,0,0.5349
3,pass_comp,G. Minshew,QB,IND,BLT,2023-09-24T17:00:00Z,20.5,over,-114,-0.480002,NaN,NaN,under,-118,0.405990,NaN,NaN,0.405990,0,0.5413
4,recv_rec,C. Hubbard,RB,CAR,SEA,2023-09-24T20:05:00Z,1.5,over,-195,-0.355043,NaN,NaN,under,143,0.282509,NaN,NaN,0.282509,0,0.4115


In [13]:
pff_df = pff_df[['propType', 'player', 'position', 'team', 'opponent', 'line', 'pff_edge', 'over_bet', 'implied_prob']]

In [14]:
for team_col in ['team', 'opponent']:
    pff_df[team_col] = pff_df[team_col].apply(utils.standardize_teams)

pff_df['week'] = week
pff_df['player'] = pff_df['player'].str.replace(r'\s+', '', regex=True)


In [20]:
pff_df = pff_df[~pff_df['propType'].str.contains('game')]
pff_df.shape

(683, 10)

In [21]:
weekly_df = nfl.import_weekly_data([year])

Downcasting floats.


In [22]:
weekly_df.head()

,player_id,player_name,player_display_name,position,position_group,headshot_url,recent_team,season,week,season_type,opponent_team,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_2pt_conversions,pacr,dakota,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr
0,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/private/f_aut...,NYJ,2023,1,REG,BUF,0,1,0.0,0,0.0,1.0,10.0,0,0,17.0,0.0,0.0,-2.031960,0,0.000000,NaN,0,0.0,0,0.0,0.0,0.0,NaN,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,0.00,0.00
1,00-0026498,M.Stafford,Matthew Stafford,QB,QB,https://static.www.nfl.com/image/private/f_aut...,LA,2023,1,REG,SEA,24,38,334.0,0,0.0,0.0,-0.0,0,0,409.0,106.0,17.0,20.679981,0,0.816626,0.217190,3,11.0,0,0.0,0.0,1.0,0.868086,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,14.46,14.46
2,00-0026498,M.Stafford,Matthew Stafford,QB,QB,https://static.www.nfl.com/image/private/f_aut...,LA,2023,2,REG,SF,34,55,307.0,1,2.0,1.0,10.0,0,0,363.0,144.0,17.0,-5.089193,0,0.845730,-0.029705,4,17.0,0,0.0,0.0,1.0,-0.438330,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,13.98,13.98
3,00-0026498,M.Stafford,Matthew Stafford,QB,QB,https://static.www.nfl.com/image/private/f_aut...,LA,2023,3,REG,CIN,18,33,269.0,1,2.0,6.0,48.0,0,0,317.0,99.0,12.0,-8.404790,0,0.848580,-0.010766,1,7.0,0,0.0,0.0,1.0,2.529576,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,11.46,11.46
4,00-0027696,J.Graham,Jimmy Graham,TE,TE,https://static.www.nfl.com/image/private/f_aut...,NO,2023,3,REG,GB,0,0,0.0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,NaN,0,NaN,NaN,0,0.0,0,0.0,0.0,0.0,NaN,0,1,1,8.0,1,0.0,0.0,8.0,0.0,1.0,2.708027,0,1.0,0.029412,0.029091,0.064481,0.0,6.80,7.80


In [23]:
weekly_df.columns

Index(['player_id', 'player_name', 'player_display_name', 'position',
       'position_group', 'headshot_url', 'recent_team', 'season', 'week',
       'season_type', 'opponent_team', 'completions', 'attempts',
       'passing_yards', 'passing_tds', 'interceptions', 'sacks', 'sack_yards',
       'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards',
       'passing_yards_after_catch', 'passing_first_downs', 'passing_epa',
       'passing_2pt_conversions', 'pacr', 'dakota', 'carries', 'rushing_yards',
       'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost',
       'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions',
       'receptions', 'targets', 'receiving_yards', 'receiving_tds',
       'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards',
       'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa',
       'receiving_2pt_conversions', 'racr', 'target_share', 'air_yards_share',
       'wopr', 'special_teams_tds', 'fantasy_points

In [24]:
weekly_df = weekly_df[weekly_df['week'] == week]
weekly_df = weekly_df[[
    'player_name',
    'position',
    'position_group', 
    'recent_team',
    'opponent_team', 
    'completions', 
    'attempts',
    'passing_yards', 
    'passing_tds', 
    'interceptions',
    'carries',
    'rushing_yards',
    'receptions', 
    'targets', 
    'receiving_yards',
    'target_share',
]]
weekly_df.head()

,player_name,position,position_group,recent_team,opponent_team,completions,attempts,passing_yards,passing_tds,interceptions,carries,rushing_yards,receptions,targets,receiving_yards,target_share
3,M.Stafford,QB,QB,LA,CIN,18,33,269.0,1,2.0,1,7.0,0,0,0.0,NaN
4,J.Graham,TE,TE,NO,GB,0,0,0.0,0,0.0,0,0.0,1,1,8.0,0.029412
5,B.Gabbert,QB,QB,KC,CHI,3,5,31.0,0,2.0,2,-1.0,0,0,0.0,NaN
6,A.Dalton,QB,QB,CAR,SEA,34,58,361.0,2,0.0,2,11.0,0,0,0.0,NaN
9,R.Cobb,WR,WR,NYJ,NE,0,0,0.0,0,0.0,0,0.0,1,2,12.0,0.071429


In [25]:
df = pd.merge(pff_df, weekly_df, how='left', left_on=['player', 'position', 'team'], right_on=['player_name', 'position', 'recent_team'])

In [26]:
df.shape

(683, 25)

In [29]:
df['rush_recv_yd'] = df['rushing_yards'] + df['receiving_yards']

In [30]:
df.sort_values('pff_edge').head()

,propType,player,position,team,opponent,line,pff_edge,over_bet,implied_prob,week,player_name,position_group,recent_team,opponent_team,completions,attempts,passing_yards,passing_tds,interceptions,carries,rushing_yards,receptions,targets,receiving_yards,target_share,rush_recv_yd
682,pass_att,M.Stafford,QB,LA,CIN,35.5,-0.037464,1,0.5169,3,M.Stafford,QB,LA,CIN,18.0,33.0,269.0,1.0,2.0,1.0,7.0,0.0,0.0,0.0,NaN,7.0
681,pass_yd,J.Love,QB,GB,NO,209.5,-0.036965,1,0.5370,3,J.Love,QB,GB,NO,22.0,44.0,259.0,1.0,1.0,9.0,39.0,0.0,0.0,0.0,NaN,39.0
680,rush_recv_yd,M.Thomas,WR,NO,GB,53.5,-0.036953,0,0.5536,3,M.Thomas,WR,NO,GB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,9.0,50.0,0.264706,50.0
679,rush_recv_yd,I.Pacheco,RB,KC,CHI,62.5,-0.036664,0,0.5370,3,I.Pacheco,RB,KC,CHI,0.0,0.0,0.0,0.0,0.0,15.0,62.0,2.0,3.0,16.0,0.081081,78.0
678,rush_yd,I.Pacheco,RB,KC,CHI,48.5,-0.036237,0,0.5392,3,I.Pacheco,RB,KC,CHI,0.0,0.0,0.0,0.0,0.0,15.0,62.0,2.0,3.0,16.0,0.081081,78.0


In [31]:
# remap columns to prop categories
df = df.rename(columns={
    'attempts': 'pass_att',
    'completions': 'pass_comp',
    'passing_yards': 'pass_yd',
    'passing_tds': 'pass_td',
    'interceptions': 'pass_int',
    'receptions': 'recv_rec',
    'receiving_yards': 'recv_yd',
    'carries': 'rush_att',
    'rushing_yards': 'rush_yd',
})

In [32]:
df.head()

,propType,player,position,team,opponent,line,pff_edge,over_bet,implied_prob,week,player_name,position_group,recent_team,opponent_team,pass_comp,pass_att,pass_yd,pass_td,pass_int,rush_att,rush_yd,recv_rec,targets,recv_yd,target_share,rush_recv_yd
0,pass_att,G.Minshew,QB,IND,BAL,32.5,0.437888,0,0.5050,3,G.Minshew,QB,IND,BAL,27.0,44.0,227.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0
1,pass_att,A.Dalton,QB,CAR,SEA,31.5,0.427262,0,0.5192,3,A.Dalton,QB,CAR,SEA,34.0,58.0,361.0,2.0,0.0,2.0,11.0,0.0,0.0,0.0,NaN,11.0
2,pass_comp,A.Dalton,QB,CAR,SEA,19.5,0.419308,0,0.5349,3,A.Dalton,QB,CAR,SEA,34.0,58.0,361.0,2.0,0.0,2.0,11.0,0.0,0.0,0.0,NaN,11.0
3,pass_comp,G.Minshew,QB,IND,BAL,20.5,0.405990,0,0.5413,3,G.Minshew,QB,IND,BAL,27.0,44.0,227.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0
4,recv_rec,C.Hubbard,RB,CAR,SEA,1.5,0.282509,0,0.4115,3,C.Hubbard,RB,CAR,SEA,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,3.0,2.0,0.052632,4.0


In [33]:
winners = []

for row in df.itertuples():
    res = False
    if getattr(row, 'over_bet'):
        res = getattr(row, getattr(row, 'propType')) > getattr(row, 'line')
    else:
        res = getattr(row, getattr(row, 'propType')) < getattr(row, 'line')
    winners.append(res)

df['win'] = winners

In [34]:
df.head()

,propType,player,position,team,opponent,line,pff_edge,over_bet,implied_prob,week,player_name,position_group,recent_team,opponent_team,pass_comp,pass_att,pass_yd,pass_td,pass_int,rush_att,rush_yd,recv_rec,targets,recv_yd,target_share,rush_recv_yd,win
0,pass_att,G.Minshew,QB,IND,BAL,32.5,0.437888,0,0.5050,3,G.Minshew,QB,IND,BAL,27.0,44.0,227.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,False
1,pass_att,A.Dalton,QB,CAR,SEA,31.5,0.427262,0,0.5192,3,A.Dalton,QB,CAR,SEA,34.0,58.0,361.0,2.0,0.0,2.0,11.0,0.0,0.0,0.0,NaN,11.0,False
2,pass_comp,A.Dalton,QB,CAR,SEA,19.5,0.419308,0,0.5349,3,A.Dalton,QB,CAR,SEA,34.0,58.0,361.0,2.0,0.0,2.0,11.0,0.0,0.0,0.0,NaN,11.0,False
3,pass_comp,G.Minshew,QB,IND,BAL,20.5,0.405990,0,0.5413,3,G.Minshew,QB,IND,BAL,27.0,44.0,227.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,False
4,recv_rec,C.Hubbard,RB,CAR,SEA,1.5,0.282509,0,0.4115,3,C.Hubbard,RB,CAR,SEA,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,3.0,2.0,0.052632,4.0,False


In [37]:
df.win.mean()

0.48462664714494874

In [43]:
df[df['pff_edge'] > 0.02].win.mean()

0.4893048128342246

In [46]:
df['decimal_odds'] = df.implied_prob.apply(utils.decimal_odds)
df['profit'] = df['win'] * df['decimal_odds']

In [54]:
df[df['pff_edge'] > 0.05].profit.mean()

0.9431130691196618

In [55]:
df[df['pff_edge'] > 0.20].profit.mean()

1.0134164633391731

In [53]:
df.head(25)

,propType,player,position,team,opponent,line,pff_edge,over_bet,implied_prob,week,player_name,position_group,recent_team,opponent_team,pass_comp,pass_att,pass_yd,pass_td,pass_int,rush_att,rush_yd,recv_rec,targets,recv_yd,target_share,rush_recv_yd,win,decimal_odds,profit
0,pass_att,G.Minshew,QB,IND,BAL,32.5,0.437888,0,0.5050,3,G.Minshew,QB,IND,BAL,27.0,44.0,227.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,False,1.980198,0.000000
1,pass_att,A.Dalton,QB,CAR,SEA,31.5,0.427262,0,0.5192,3,A.Dalton,QB,CAR,SEA,34.0,58.0,361.0,2.0,0.0,2.0,11.0,0.0,0.0,0.0,NaN,11.0,False,1.926040,0.000000
2,pass_comp,A.Dalton,QB,CAR,SEA,19.5,0.419308,0,0.5349,3,A.Dalton,QB,CAR,SEA,34.0,58.0,361.0,2.0,0.0,2.0,11.0,0.0,0.0,0.0,NaN,11.0,False,1.869508,0.000000
3,pass_comp,G.Minshew,QB,IND,BAL,20.5,0.405990,0,0.5413,3,G.Minshew,QB,IND,BAL,27.0,44.0,227.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,False,1.847404,0.000000
4,recv_rec,C.Hubbard,RB,CAR,SEA,1.5,0.282509,0,0.4115,3,C.Hubbard,RB,CAR,SEA,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,3.0,2.0,0.052632,4.0,False,2.430134,0.000000
5,recv_rec,D.Smythe,TE,MIA,DEN,2.5,0.253962,0,0.4425,3,D.Smythe,TE,MIA,DEN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,15.0,0.035714,15.0,True,2.259887,2.259887
6,recv_rec,J.Conner,RB,ARI,DAL,2.5,0.244922,0,0.4464,3,J.Conner,RB,ARI,DAL,0.0,0.0,0.0,0.0,0.0,14.0,98.0,2.0,2.0,18.0,0.095238,116.0,True,2.240143,2.240143
7,recv_rec,C.Otton,TE,TB,PHI,2.5,0.233494,0,0.4545,3,C.Otton,TE,TB,PHI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,16.0,0.080000,16.0,True,2.200220,2.200220
8,pass_yd,G.Minshew,QB,IND,BAL,215.5,0.232178,0,0.5370,3,G.Minshew,QB,IND,BAL,27.0,44.0,227.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,False,1.862197,0.000000
9,recv_rec,K.Bourne,WR,NE,NYJ,2.5,0.231446,0,0.4386,3,K.Bourne,WR,NE,NYJ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,5.0,46.0,0.185185,46.0,False,2.279982,0.000000
